## Initialization

Allows us to control instance and class creation.

* `__new__`
* `__init__`
* `__del__`


### Initialization

In [3]:
class A:
    pass

In [4]:
a = A()

In [5]:
a.x = 1

In [6]:
a.x

1

We can assign values to attributes w/o a init, but that's rarely useful.

In [7]:
class A:
    def __init__(self):
        self.x = 1

In [8]:
a = A()

In [9]:
a.x

1

Init allows to setup an instance.

* we can have instance only attributes
* by default, if attributes are not found on the instance, they are searched for in the class and superclasses
* instance attributes belong to an instance, class attributes are shared among all instances

Example: A class that knows how often it has been instantiated.

In [20]:
class Counted:
    count = 0
    
    def __init__(self):
        Counted.count += 1 # self would add count to instance, not class
        self.x = 1

In [21]:
a = Counted()

In [22]:
b = Counted()

In [23]:
b.count

2

At init time, we already have an instance of the class, passed in as `self` (just as in all other methods).

### Finalizer

* less commonly used, `__del__`

In [27]:
class A:
    def __del__(self):
        print("calling __del__")

In [29]:
x = [A()]

In [30]:
x = None

calling __del__


### Class creation

* with `__new__` we can hook into instance creation
* `__new__` is called before `__init__`

In [43]:
class A:
    def __new__(cls):
        print("__new__")
    def __init__(self):
        print("__init__")


In [44]:
a = A() # no init!

__new__


* `__new__` takes the class (of which we're preparing an instance) as first argument
* needs to return something, otherwise nothing is returned

In [45]:
a is None

True

> The return value of `__new__()` should be the new object instance (usually an instance of cls).

In [46]:
class A:
    def __new__(cls):
        return cls()
    def __init__(self):
        print("__init__")

In [49]:
# a = A() # Does this work?


> RecursionError: maximum recursion depth exceeded while calling a Python object

Instanciating the class will call `__new__` again.


> Typical implementations create a new instance of the class by invoking the superclass’s `__new__()` method using `super().__new__(cls[, ...])` with appropriate arguments and then modifying the newly created instance as necessary before returning it.

In [50]:
class A:
    def __new__(cls):
        print(cls, type(cls), cls.__bases__)
    def __init__(self):
        print("__init__")

In [51]:
a = A()

<class '__main__.A'> <class 'type'> (<class 'object'>,)


In this simple case, we could write.

In [55]:
class A:
    def __new__(cls):
        print("__new__")
        return object.__new__(cls)
    def __init__(self):
        print("__init__")

In [56]:
a = A()

__new__
__init__


In [57]:
type(a)

__main__.A

Typically, we use `super` to cater for inheritance.

In [66]:
class A:
    def __new__(cls):
        print("__new__")
        return super().__new__(cls)
    def __init__(self):
        print("__init__")

In [67]:
a = A()

__new__
<super: <class 'A'>, <A object>> <class 'super'>
__init__


Sidenote: what does super do here?
    
* [super](https://docs.python.org/3/library/functions.html#super) is a built-in

> Return a proxy object that delegates method calls to a parent or sibling class of type.

We use it to not hard-code super classes.

> There are two typical use cases for super. In a class hierarchy with single inheritance, super can be used to refer to parent classes without naming them explicitly, thus making the code more maintainable.

Also used for multiple inhertance.

> The second use case is to support cooperative multiple inheritance in a dynamic execution environment. This use case is unique to Python and is not found in statically compiled languages or languages that only support single inheritance. This makes it possible to implement “diamond diagrams” where multiple base classes implement the same method.

Usage:

```python
class C(B):
    def method(self, arg):
        super().method(arg)    # This does the same thing as:
                               # super(C, self).method(arg)
```

When used outside of a class, we need to be explicit.

>  The zero argument form only works inside a class definition, as the compiler fills in the necessary details to correctly retrieve the class being defined, as well as accessing the current instance for ordinary methods.

In [82]:
class Map(dict):
    a = 1

m = Map()
s = super(Map, m)
print(s.__doc__)

dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [83]:
class A(dict):
    """ A """

class B(A):
    """ B """

class Map(B):
    """ Map """
    
m = Map()
s = super(Map, m)
print(s.__doc__)

 B 


Here, we could also use the one argument variant.

In [89]:
class A(dict):
    """ A """

class B(A):
    """ B """

class Map(B):
    """ Map """
    
m = Map()
s = super(Map) # RuntimeError: super(): no arguments - if not args supplied
print(s) # unbound

<super: <class 'Map'>, NULL>


In [91]:
print(s.__get__(m).__doc__)

 B 


Back to `__new__`

In [96]:
class A:
    def __new__(cls):
        print("__new__")
        inst = super().__new__(cls)
        print(inst, type(inst))
        return inst
    def __init__(self):
        print("__init__")

In [97]:
a = A()

__new__
<__main__.A object at 0x7f820ed6d210> <class '__main__.A'>
__init__


We can also build object with the builtin type function.

> With three arguments, return a new type object. This is essentially a dynamic form of the class statement. 

In [98]:
type?

In [99]:
A = type("A", (object,), {})

In [100]:
A

__main__.A

In [101]:
a = A()

We could get populate the dictionary, too.

In [102]:
A = type("A", (object,), {"x": 1})

In [103]:
a = A()

In [104]:
a.x

1

In essence:
    
* with `__new__` we run custom code for all instances

Also, `__new__` is a static method on object.

In [109]:
# object.__new__(dict, None, None) # TypeError
# TypeError: object.__new__(dict) is not safe, use dict.__new__()

In [110]:
object.__new__(str, None, None)

TypeError: object.__new__(str) is not safe, use str.__new__()

### Another example

In [129]:
class A:
    def __new__(cls, *args, **kwargs):
        return super().__new__(A) 
    def __init__(self, name, location="world"):
        self.name = name
        self.location = location

In [127]:
a = A("abc")

('abc',) {}


In [128]:
a.name

'abc'

In [130]:
import datetime

In [169]:
class A:
    """ A """
    def __new__(cls, *args, **kwargs):
        inst = super().__new__(A)
        inst.__doc__ = inst.__doc__ + f" -- instatiated at {datetime.datetime.now()}"
        return inst
    def __init__(self, name, location="world"):
        self.name = name
        self.location = location

In [170]:
a = A("abc")

In [171]:
help(a)

Help on A in module __main__:

<__main__.A object>
    A  -- instatiated at 2022-12-07 21:30:22.434754



### Use cases for `__new__`

* typically create new instance and prepare it
* keeping track of instances (e.g. limits)

In [172]:
class Limited:
    _active = 0
    
    def __new__(cls, *args, **kwargs):
        inst = super().__new__(cls)
        def __del__(self):
            cls._active -= 1
        cls.__del__ = __del__
        cls._active += 1
        if cls._active > 2:
            raise ValueError("only two instances allowed")
        return inst
    
    def __init__(self):
        pass

In [173]:
# a = Limited()
# b = Limited()
# c = Limited() # ValueError: only two instances allowed

In [162]:
a = Limited()
b = Limited()
b = None
c = Limited()